# Problem: Lemonade Change

Statement
There is a lemonade stand where customers can buy one lemonade at a time for 
$
5
$5
 and pay with a 
$
5
$5
, 
$
10
$10
, or 
$
20
$20
 bill. It is necessary to return the correct change to each customer so that the net transaction is completed successfully with a total payment of 
$
5
$5
. Note that no change is available initially.

Given an integer array, bills, where bills[i] represents the bill paid by the 
i
t
h
i 
th
 
 customer, return TRUE if it is possible to provide every customer with the correct change, or FALSE otherwise.

Constraints:

1
≤
1≤
 bills.length 
≤
500
≤500

bills[i] is either 
5
5
, 
10
10
, or 
20
20
.

Hint:
- You might want to go over the Greedy Techniques pattern again.

In [ ]:
from collections import defaultdict

def lemonade_change(bills):
    available_bills = defaultdict(int)
    
    for bill in bills:
        if bill == 5:
            available_bills[5] += 1
        
        elif bill == 10:
            if available_bills[5] == 0:
                return False
            available_bills[5] -= 1
            available_bills[10] += 1
        
        elif bill == 20:
            if available_bills[10] > 0 and available_bills[5] > 0:
                available_bills[10] -= 1
                available_bills[5] -= 1
            elif available_bills[5] >= 3:
                available_bills[5] -= 3
            else:
                return False
    
    return True


In [5]:
bills = [5,5,5,5,20,10,10]
lemonade_change(bills)

False

In [ ]:
# Better optimized solution
def lemonade_change(bills):
    five, ten = 0, 0   # counters for $5 and $10 bills we have
    
    for bill in bills:
        if bill == 5:
            five += 1
        elif bill == 10:
            if five == 0:
                return False
            five -= 1
            ten += 1
        else:  # bill == 20
            if ten > 0 and five > 0:
                ten -= 1
                five -= 1
            elif five >= 3:
                five -= 3
            else:
                return False
    
    return True
